## Problem

You need to create a prototype pricing model that can go through further validation and testing before being put into production. Eventually, this model may be the basis for fully automated quoting to clients, but for now, the desk will use it with manual oversight to explore options with the client. 

You should write a function that is able to use the data you created previously to price the contract. The client may want to choose multiple dates to inject and withdraw a set amount of gas, so your approach should generalize the explanation from before. Consider all the cash flows involved in the product.

The input parameters that should be taken into account for pricing are:

- Injection dates. 
- Withdrawal dates.
- The prices at which the commodity can be purchased/sold on those dates.
- The rate at which the gas can be injected/withdrawn.
- The maximum volume that can be stored.
- Storage costs.

In [1]:
import pandas as pd
from statsmodels.tsa.forecasting.stl import STLForecast
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime
from dateutil import relativedelta

We use the same model from task 1 as the starting point for the dataset and the extrapolated data.

In [2]:
df = pd.read_csv("Nat_Gas.csv", parse_dates=["Dates"])
df = df.set_index("Dates")
stlf = STLForecast(df, ARIMA, model_kwargs={"order": (1, 1, 0), "seasonal_order": (1, 1, 0, 12), "freq": "M"})
model = stlf.fit()
forecasts = model.forecast(steps=12)
forecasts = pd.DataFrame(forecasts)
forecasts.columns = ["Prices"]
nat_gas_pricing = pd.concat([df, forecasts])
nat_gas_pricing = nat_gas_pricing.round(2)
nat_gas_pricing

c:\Users\ROG ZEPHYRUS\miniconda3\envs\mds574\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


,Prices
2020-10-31,10.10
2020-11-30,10.30
2020-12-31,11.00
2021-01-31,10.90
2021-02-28,10.90
2021-03-31,10.90
2021-04-30,10.40
2021-05-31,9.84
2021-06-30,10.00
2021-07-31,10.10


In [3]:
### HELPER FUNCTIONS

def price_estimate(date, natural_gas_price_data=nat_gas_pricing):
    """
    Given a date, return the price of natural gas.
    :param date: str (in the form %Y-%m)
    :param natural_gas_price_data: pd.DataFrame
    :return: float
    """
    date_idx = natural_gas_price_data.index.strftime("%Y-%m").tolist()
    try:
        idx = date_idx.index(date)
        return natural_gas_price_data.iloc[idx].values[0]
    except ValueError:
        print(f"No predicted value for date: {date}")
        return -1
    
def get_month_difference(date_1, date_2):
    d1 = datetime.strptime(date_1, "%Y-%m")
    d2 = datetime.strptime(date_2, "%Y-%m")
    r = relativedelta.relativedelta(d2, d1)
    return r.months

In [4]:
def get_contract_price(
    injection_dates,
    withdraw_dates,
    commodity_prices,
    rates,
    max_storage_volume,
    storage_cost
):
    """
    Given a list of buy and sell dates and the units of commodity bought, return the price of the contract when executing the buy and sells. 
    Assumes that any purchases are before sells and that we do not make decisions that do not make sense (e.g. buying without selling at a later point),
    and that the buy and sell dates are sorted chronologically.
    The total amount of commodity purchased cannot exceed the maximum storage volume.
    :param injection_dates: list
    :param withdraw_dates: list
    :param commodity_prices: pd.DataFrame
    :param rates: list
    :param max_storage_volume: float
    :param storage_cost: float
    """
    if len(injection_dates) + len(withdraw_dates) != len(rates):
        raise ValueError("The total number of injections/withdrawals should be equal to the total rates that is being injected/withdrawn.")
    if injection_dates[0] > withdraw_dates[0]:
        raise ValueError("The first injection date should be before the first withdrawal date.")
    contract_price = 0
    injection_idx = 0
    withdraw_idx = 0
    stored_commodity = 0
    prev_date = injection_dates[0]
    while injection_idx < len(injection_dates) and withdraw_idx < len(withdraw_dates):
        rate_idx = injection_idx + withdraw_idx
        if injection_dates[injection_idx] <= withdraw_dates[withdraw_idx]:
            # inject/purchase commodity
            stored_commodity += rates[rate_idx]
            if stored_commodity > max_storage_volume:
                raise ValueError("Exceeded maximum storage capacity. Contract will not work.")
            contract_price -= rates[rate_idx] * price_estimate(injection_dates[injection_idx], commodity_prices)
            injection_idx += 1
        else:
            # withdraw/sell commodity
            stored_commodity -= rates[rate_idx]
            # print(f"Sell price: {(rates[rate_idx] * price_estimate(withdraw_dates[withdraw_idx], commodity_prices))}")
            contract_price = (
                contract_price 
                + (rates[rate_idx] * price_estimate(withdraw_dates[withdraw_idx], commodity_prices))
                - (storage_cost * get_month_difference(prev_date, withdraw_dates[withdraw_idx]))
            )
            prev_date = withdraw_dates[withdraw_idx]
            withdraw_idx += 1
        # print(f"Current contract: {contract_price}")
    if withdraw_idx < len(withdraw_dates):
        # all injections/purchases complete, remaining are withdraws/sell
        for i in range(withdraw_idx, len(withdraw_dates)):
            rate_idx = injection_idx + i
            stored_commodity -= rates[rate_idx]
            # print(f"Sell price: {(rates[rate_idx] * price_estimate(withdraw_dates[i], commodity_prices))}")
            contract_price = (
                contract_price 
                + (rates[rate_idx] * price_estimate(withdraw_dates[i], commodity_prices)) 
                - (storage_cost * get_month_difference(prev_date, withdraw_dates[i]))
            )
            prev_date = withdraw_dates[withdraw_idx]
            # print(f"Current contract: {contract_price}")
    return contract_price
        

In [5]:
get_contract_price(
    injection_dates=["2023-07"],
    withdraw_dates=["2024-02"],
    commodity_prices=nat_gas_pricing,
    rates=[100, 100],
    max_storage_volume=500,
    storage_cost=1
)

93.0

In [6]:
get_contract_price(
    injection_dates=["2023-07"],
    withdraw_dates=["2024-02", "2025-03"],
    commodity_prices=nat_gas_pricing,
    rates=[100, 50, 50],
    max_storage_volume=500,
    storage_cost=1
)

128.5

In [7]:
get_contract_price(
    injection_dates=["2024-09"],
    withdraw_dates=["2025-03"],
    commodity_prices=nat_gas_pricing,
    rates=[10, 10],
    max_storage_volume=100,
    storage_cost=1
)

7.300000000000011

In [8]:
# Should raise an error because the maximum storage capacity is exceeded
try:
    get_contract_price(
        injection_dates=["2024-09"],
        withdraw_dates=["2025-03"],
        commodity_prices=nat_gas_pricing,
        rates=[110, 10],
        max_storage_volume=100,
        storage_cost=1
)
except ValueError:
    print("ValueError raised due to exceeding maximum storage capacity.")

ValueError raised due to exceeding maximum storage capacity.
